In [2]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from math import sqrt
from sklearn.model_selection import KFold

In [61]:
train_df = []
for idx in range(0, 10):
    if(idx == 0):
        continue
    train_df += [kfold[idx]]
    
train_df = pd.concat(train_df, ignore_index = True)
len(train_df['name'])
for idx in range(0, len(train_df['name'])):
    print train_df['name'][idx]

Disneyland
#boostyourrun
LUV
latin
JAMS
~Rando~
Party Rock!!
party people
The Piano Guys
Worship
Brasileiras
country
4/20
2016
Willie
beach
Gold Digger
Fall 2015
dance it out
greek
All the songs
Kyle
HALLOWEEN
w o r k o u t
party music
summer
XXXMas
BBG
bae
oldies
its lit
go to
Killin' It
October 2017
Rap/Pop
1990's 
Rap
GUACAMOLE
14
That New New
Relaxing
sleep
Get It!
bumps
picks
🤘🏼
Mashup
Ladies
Alt Jams
Reggae/Ska
Top hits
jazzy 
Pop
Emotional
Fall
ouch
Love Music
Bang Bang 
Solitude
feels
Silence
English 
Blues
pump
volleyball playlist 
september 
motown
folky
Rock alternativo
Summer Chill
Rap Classics
My favs
motivational 
Up Beat
Wes Anderson
Love
basic
Another
LIT!!!!!!!
YEET
🤤🤤
Summer Playlist
high
Music
Chill Alternative
feelz
Alexia
Mellow.
Britt
Christian Songs
Kendrick Lamar 
jesus jams
Daddy Issues
🔥🔥🔥🔥
sb2k17
Dance
2013
Spring16
soul
Best Songs of 2016
Ladies
Love 
Cash
#Workout
guitar.
running 
Shower
Saturn
Christmas music
Bus playlist
Xmas
My music
Latino
Mumford & Son

In [73]:
kfold[1]['name'].reset_index()

,index,name
0,100,Disneyland
1,101,#boostyourrun
2,102,LUV
3,103,latin
4,104,JAMS
5,105,~Rando~
6,106,Party Rock!!
7,107,party people
8,108,The Piano Guys
9,109,Worship


In [80]:
f = open('mpd.slice.0-999.json').read()
data = json.loads(f)
playlists = data["playlists"]
df = pd.DataFrame(playlists)
#print df
k = 10
topic_num = 7
kfold = [df[i:i + len(df)/k] for i in xrange(0, len(df), len(df)/k)]

max_f1score = 0
min_f1score = 1
average_f1score = 0
for idx in range(0, k):
    
    test_df = kfold[idx].reset_index()
    test_texts =  [[word for word in name.lower().split()]for name in list(set(kfold[idx]['name']))]
    
    train_texts = []
    train_df = []
    for tidx in range(0, k):
        if(tidx == idx):
            continue
        train_df += [kfold[tidx]]
        train_texts.extend(list(set(kfold[tidx]['name'])))
    train_texts = [[word for word in name.lower().split()]for name in train_texts]
    train_df = pd.concat(train_df, ignore_index = True)
    
    dictionary = corpora.Dictionary(train_texts)
    corpus = [dictionary.doc2bow(text) for text in train_texts]
    model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
                                    num_topics = topic_num, random_state = 0,
                                    chunksize = 2000, passes = 10)
    
    train_dic = {}
    for s in range(0, len(train_texts)):
        t = train_texts[s]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        train_dic[name] = dict(get_document_topics)
    train_dic = pd.DataFrame(train_dic)
    
    test_dic = {}
    for s in range(0, len(test_texts)):
        t = test_texts[s]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        test_dic[name] = dict(get_document_topics)
    test_dic =pd.DataFrame(test_dic).T
    
    result = []
    for trainName in train_dic.columns:
        idx = 0
        c = 0
        columns_ones = []
        for testName in test_dic.index:
            for idx in range(0, topic_num):
                c += pow(train_dic[trainName][idx] - test_dic[idx][testName], 2)
            c = 1/(1+sqrt(c))
            columns_ones.append(c)
        result.append(columns_ones)
    #train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
    result = pd.DataFrame(result, columns = test_dic.index, index = train_dic.columns)
    
    count = 0
    precision = 0
    recall = 0
    f1socre = 0
    for name in result.columns:
        #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
        result_playlist_name = result[name].sort_values(ascending = False).head(10).index
        #print count

        #test_playlist 생성
        test_playlist = []
        for i in range(0, len(test_df['name'])):
            if(test_df['name'][i].lower() == name):
                test_track_uri = pd.DataFrame(test_df['tracks'][i])
                test_playlist = list(test_track_uri['track_uri'])
        #print "test_playlist length :{}".format(len(test_playlist))        
        if(len(test_playlist) == 0):
            continue

            #비교할 train_playlist 생성
        train_playlist = []
        for train_name in result_playlist_name:
            for i in range(0,len(train_df['name'])):
                
                if(train_df['name'][i].lower() == train_name):
                    df = pd.DataFrame(train_df['tracks'][i])
                    train_playlist += list(df['track_uri'])
        #print "train_playlist length : {}".format(len(train_playlist))            
        precision += 1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(train_playlist))
        recall += 1.0 * len(set(train_playlist).intersection(set(test_playlist)))/len(set(test_playlist))
        count+=1
        #print count
    precision = 1.0 * precision/(count)
    recall = 1.0 * recall /(count)
    f1score = (2 * precision * recall) / (precision + recall)
    print "precision : {}\nrecall : {}\nf1score : {}".format(precision,recall,f1score)
    print
    if(min_f1score > f1score):
        min_f1score = f1score
    if(max_f1score < f1score):
        max_f1score = f1score
    
    average_f1score += f1score

    
average_f1score = average_f1score/10
print "minF1score : {}, maxF1score: {}, averageF1score: {}".format(min_f1score, max_f1score, average_f1score)

precision : 0.00540244418183
recall : 0.0928576941547
f1score : 0.0102108244099

precision : 0.0054902019609
recall : 0.0711556034154
f1score : 0.0101938685746

precision : 0.00558596263718
recall : 0.105375664444
f1score : 0.0106095150177

precision : 0.00489667545051
recall : 0.0788370350895
f1score : 0.00922064415452

precision : 0.00591070100412
recall : 0.0748070787492
f1score : 0.0109557591111

precision : 0.00439123982054
recall : 0.0656660706827
f1score : 0.00823198785021

precision : 0.00565949387649
recall : 0.0922424275584
f1score : 0.0106646620673

precision : 0.00509270119748
recall : 0.0818310774003
f1score : 0.00958865876726

precision : 0.00587907703388
recall : 0.0816830734802
f1score : 0.0109686908906

precision : 0.00510904181512
recall : 0.0776760489699
f1score : 0.0095874795439

minF1score : 0.00823198785021, maxF1score: 0.0109686908906, averageF1score: 0.0100232090387


In [76]:
f = open('mpd.slice.0-999.json').read()
data = json.loads(f)
playlists = data["playlists"]
df = pd.DataFrame(playlists)
#print df
k = 10
topic_num = 10
kfold = [df[i:i + len(df)/k] for i in xrange(0, len(df), len(df)/k)]

max_f1score = 0
min_f1score = 1
average_f1score = 0
for idx in range(0, k):
    
    test_df = kfold[idx].reset_index()
    test_texts =  [[word for word in name.lower().split()]for name in list(set(kfold[idx]['name']))]
    
    train_texts = []
    train_df = []
    for tidx in range(0, k):
        if(tidx == idx):
            continue
        train_df += [kfold[tidx]]
        train_texts.extend(list(set(kfold[tidx]['name'])))
    train_texts = [[word for word in name.lower().split()]for name in train_texts]
    train_df = pd.concat(train_df, ignore_index = True)
    
    dictionary = corpora.Dictionary(train_texts)
    corpus = [dictionary.doc2bow(text) for text in train_texts]
    model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
                                    num_topics = topic_num, random_state = 0,
                                    chunksize = 2000, passes = 10)
    
    train_dic = {}
    for s in range(0, len(train_texts)):
        t = train_texts[s]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        train_dic[name] = dict(get_document_topics)
    train_dic = pd.DataFrame(train_dic)
    
    test_dic = {}
    for s in range(0, len(test_texts)):
        t = test_texts[s]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        test_dic[name] = dict(get_document_topics)
    test_dic =pd.DataFrame(test_dic).T
    
    result = []
    for trainName in train_dic.columns:
        idx = 0
        c = 0
        columns_ones = []
        for testName in test_dic.index:
            for idx in range(0, topic_num):
                c += pow(train_dic[trainName][idx] - test_dic[idx][testName], 2)
            c = 1/(1+sqrt(c))
            columns_ones.append(c)
        result.append(columns_ones)
    #train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
    result = pd.DataFrame(result, columns = test_dic.index, index = train_dic.columns)
    
    count = 0
    precision = 0
    recall = 0
    f1socre = 0
    for name in result.columns:
        #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
        result_playlist_name = result[name].sort_values(ascending = False).head(10).index
        #print count

        #test_playlist 생성
        test_playlist = []
        for i in range(0, len(test_df['name'])):
            if(test_df['name'][i].lower() == name):
                test_track_uri = pd.DataFrame(test_df['tracks'][i])
                test_playlist = list(test_track_uri['track_uri'])
        #print "test_playlist length :{}".format(len(test_playlist))        
        if(len(test_playlist) == 0):
            continue

            #비교할 train_playlist 생성
        train_playlist = []
        for train_name in result_playlist_name:
            for i in range(0,len(train_df['name'])):
                
                if(train_df['name'][i].lower() == train_name):
                    df = pd.DataFrame(train_df['tracks'][i])
                    train_playlist += list(df['track_uri'])
                    
        precision += 1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(train_playlist)) *100
        recall += 1.0 * len(set(train_playlist).
                                intersection(set(test_playlist)))/len(set(test_playlist))*100
        count+=1
    precision = 1.0 * precision/(count)
    recall = 1.0 * recall /(count)
    f1score = (2 * precision * recall) / (precision + recall)
    print "precision : {}\nrecall : {}\nf1score : {}".format(precision,recall,f1score)
    print
    if(min_f1score > f1score):
        min_f1score = f1score
    if(max_f1score < f1score):
        max_f1score = f1score
    
    average_f1score += f1score

    
average_f1score = average_f1score/10
print "minF1score : {}, maxF1score: {}, averageF1score: {}".format(min_f1score, max_f1score, average_f1score)

precision : 0.450537705747
recall : 6.09147917376
f1score : 0.839019862559

precision : 0.461738475576
recall : 5.36449956
f1score : 0.850289958611

precision : 0.519073416823
recall : 7.79858780116
f1score : 0.97336006126

precision : 0.544131208098
recall : 7.16981578814
f1score : 1.01149788261

precision : 0.544610626834
recall : 5.17830390129
f1score : 0.985567518004

precision : 0.469476053197
recall : 7.66770216423
f1score : 0.884779085074

precision : 0.576038307775
recall : 6.46846985472
f1score : 1.0578698592

precision : 0.452929926277
recall : 5.95651476589
f1score : 0.841846344997

precision : 0.597040898036
recall : 5.78675319825
f1score : 1.08240594044

precision : 0.534634387689
recall : 7.1783938999
f1score : 0.995151601723

minF1score : 0.839019862559, maxF1score: 1.08240594044, averageF1score: 0.952178811449
